# 03-topic-modelling-times

Calculate the topic distribution for every article in every week, over a range of years.
So first I'll put all the documents together, then calculate the words-per-topic matrix, then run a long model, and finally produce the documents-per-topic matrix.

In [3]:
import os
import glob
import string
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
import time

import pandas as pd
import numpy as np
from functools import reduce
from scipy.stats import entropy
from datetime import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("poster", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.mpl.rc("figure", figsize=(9,6))

import warnings; warnings.filterwarnings('ignore')

import spacy
nlp = spacy.load('en')

In [ ]:
"""
all_article_types = pd.read_csv('scraping_times/data/all_article_types.csv')

all_article_types = all_article_types.set_index([
                         [ x[:10] for x in all_article_types.filename ],
                         all_article_types.filename])


df_allnews = pd.DataFrame()
OCR_folder = 'scraping_times/data/OCRtext/'
columns = ['pubdate', 'headline', 'text', 'id', 'type_article']

min_file_size = 1000 # bytes
yearrange = range(1947,1950)

for year in yearrange:
    all_infiles = glob.glob(OCR_folder + str(year) + '*/*')
    to_df = []
    for infile in all_infiles:
        file_size = os.stat(infile).st_size # in bytes
        if file_size > min_file_size:
            
            with open(infile) as f:
                lines = [ i[:-1] for i in f.readlines()[2:] ]
                headline = lines[0]
                pubdate = infile.split('/')[3]
                text = ' '.join(lines[1:])
                filename = pubdate+'/'+infile.split('/')[-1]
                type_article = all_article_types.loc[pubdate].loc[filename]['type_article']
              
                if len(text) > 50:
                    to_df += [( pubdate, headline, text, filename, type_article )]
                    
    print("%d newspieces in %d" % (len(to_df),year))                
    df_allnews = df_allnews.append(pd.DataFrame(to_df,columns=columns),ignore_index=True)
    
print("%d newspieces from %d to %d" % (len(df_allnews), min(yearrange), max(yearrange)) )
df_allnews.head()
"""

In [2]:
df_allnews = pd.DataFrame()
OCR_folder = 'scraping_times/data/OCRtext/'
columns = ['pubdate', 'headline', 'text', 'id']

min_file_size = 1000 # bytes
yearrange = range(1947,2013)

for year in yearrange:
    all_infiles = glob.glob(OCR_folder + str(year) + '*/*')
    to_df = []
    for infile in all_infiles:
        file_size = os.stat(infile).st_size # in bytes
        if file_size > min_file_size:
            
            with open(infile) as f:
                lines = [ i[:-1] for i in f.readlines()[2:] ]
                headline = lines[0]
                pubdate = infile.split('/')[3]
                text = ' '.join(lines[1:])
                filename = pubdate+'/'+infile.split('/')[-1]
              
                if len(text) > 50:
                    to_df += [( pubdate, headline, text, filename )]
                    
    print("%d newspieces in %d" % (len(to_df),year))                
    df_allnews = df_allnews.append(pd.DataFrame(to_df,columns=columns),ignore_index=True)
    
print("%d newspieces from %d to %d" % (len(df_allnews), min(yearrange), max(yearrange)) )
df_allnews.head()

4161 newspieces in 1947
3290 newspieces in 1948
4342 newspieces in 1949
4535 newspieces in 1950
4664 newspieces in 1951
4989 newspieces in 1952
5669 newspieces in 1953
5286 newspieces in 1954
5375 newspieces in 1955
5305 newspieces in 1956
5378 newspieces in 1957
5857 newspieces in 1958
5975 newspieces in 1959
6435 newspieces in 1960
7221 newspieces in 1961
7726 newspieces in 1962
7519 newspieces in 1963
7493 newspieces in 1964
7389 newspieces in 1965
6642 newspieces in 1966
6914 newspieces in 1967
7461 newspieces in 1968
9087 newspieces in 1969
7945 newspieces in 1970
7662 newspieces in 1971
7790 newspieces in 1972
7721 newspieces in 1973
7378 newspieces in 1974
7459 newspieces in 1975
7766 newspieces in 1976
9218 newspieces in 1977
7072 newspieces in 1978
901 newspieces in 1979
6896 newspieces in 1980
6764 newspieces in 1981
6260 newspieces in 1982
6593 newspieces in 1983
5781 newspieces in 1984
6374 newspieces in 1985
7212 newspieces in 1986
8067 newspieces in 1987
7961 newspieces i

,pubdate,headline,text,id
0,1947-04-30,Two Minutes,"The thought that there is a list of 450,000 pe...",1947-04-30/068_GALE_CS84624971.txt
1,1947-04-30,&USTRALIAN HEAT WAVE,-DEATHS HIGHER TEMPERATURES FORECAST FROM OUR ...,1947-04-30/143_GALE_CS135618086.txt
2,1947-04-30,"4366,000,000 SAVINGS",TARGET LORD MONTGOMERY ON WINNING PROSPERITY F...,1947-04-30/102_GALE_CS119096478.txt
3,1947-04-30,FINANCIAL TALKS WrM,- PAKISTAN RELEASE OF BALANCES Financial negot...,1947-04-30/037_GALE_CS36391090.txt
4,1947-04-30,FATEFUL YEAR FOR CEYLON,SELF-GOVERNMENT UNDER THE SOULBURY SCHEME A LA...,1947-04-30/076_GALE_CS85279902.txt


In [3]:
print("Some ads are still classified as news:")
df_allnews[df_allnews.pubdate=='2011-07-20'].iloc[0].values

Some ads are still classified as news:


array(['2011-07-20', 'Beauty and the beach: ten great products',
       'Murad sunblock Invest in quality sunscreen to prevent premature ageing and, of course, sunburn! Murad pomphenol sunguard dietary supplement Protect your skin from the inside with this clever pomegranate supplement COWSHED i..;|-jnS UwJy \'it\'ttoo Mop basil mint shampoo Give your hair a holiday: wash out impurities caused by pollution and styling products with this fresh shampoo Mop C-System moisture complex for conditioning and detangling Sun and chlorine can dry out your hair, so give it a treat with a rich conditioner t JF3U m Cowshed lazy cow soothing body lotion A luxurious- smelling lotion that\'s perfect for calming sun-exposed skin »«((*»» wgs-\'il; piMMigt* mop- St Tropez wash-off instant shimmer stick Ifthe weather lets you down, create a tan with this handy instant bronzer A? \'-A? - > A * s4AAk " "* ^ ^ ^y? » » <■ * A *x, fi i. * f f m A W\'l s ^=^^ . Bellapierre compact foundation Choose a compact a l

In [4]:
import re
def has_letter(s):
    return ( re.search('[a-zA-Z]', s) != None )

def tokenize(text0):
    #stopwords = set(nltk.corpus.stopwords.words('english'))

    from spacy.lang.en.stop_words import STOP_WORDS

    stopwords = STOP_WORDS
    for word in ['year','new','said']:
        stopwords.add(word)
    
    minlength = 3
    
    invalidChars = { '¡', '§', '©', '\xad', '°', '²', '³', 'µ', '¹', '¿', '×', '\u200b', 
                    '•', '‣', '…', '⁄', '₂', '€', '™', '▇', '■', '▶', '◆', '●', '★', '✽',
                    '❏', '➝', '主', '原', '年', '後', '歸', '物', '舧', '舰'}
    invalidChars = invalidChars.union(set(string.punctuation.replace("-", "–„“")))
    
    text = text0.replace('- ','') # for split words
    
    for token in nltk.word_tokenize(text):
        t = token.lower()
        if (len(t)<minlength) or (t in stopwords) \
        or (t in string.punctuation) or (t[0] in string.punctuation) \
        or any(char in invalidChars for char in token) \
        or not has_letter(t):
            continue
        yield t
        
def normalise(vec):
    return vec / np.dot(vec,vec)

def combine_vectors(vectors):
    return normalise(np.sum(vectors, axis=0))

def important_words(vectorizer, vec, n):
    return sorted(zip(vectorizer.get_feature_names(), vec), key=lambda x:x[1], reverse=True)[:n]

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
start = time.time()

# list of text documents
text    = df_allnews.text.values
doc_ids = df_allnews.id.values

print("Listed documents")

# create the transform
vectorizer = CountVectorizer(tokenizer=tokenize, min_df=0.005, max_df=0.8)

print("Created the transform")

# tokenize and build vocab
vectorizer.fit(text)

print("Ran vectorizer.fit(text)")

# summarize
#print(vectorizer.vocabulary_)

# encode document-term matrix
dtm = vectorizer.transform(text)

Listed documents
Created the transform
Ran vectorizer.fit(text)


In [6]:
# summarize encoded vector
print('Shape of document-term matrix (documents, tokens):', dtm.shape)
print('Total number of tokens:', dtm.sum() )
#print(type(dtm))
#print(dtm.toarray())

end = time.time()
print(round((end - start)/60.0,2),'minutes')

from scipy.sparse import save_npz
save_npz('times_data/dtm_matrix.npz', dtm)

Shape of document-term matrix (documents, tokens): (455349, 6033)
Total number of tokens: 85821427
103.07 minutes


In [16]:
from scipy.sparse import load_npz
dtm = load_npz('times_data/dtm_matrix.npz')

dtm

<455349x6033 sparse matrix of type '<class 'numpy.int64'>'
	with 57260404 stored elements in Compressed Sparse Row format>

In [17]:
nnz_per_row = dtm.getnnz(axis=1)
non_null_rows = np.where(nnz_per_row > 0)[0]
null_rows     = np.where(nnz_per_row <= 0)[0]

dtm = dtm[dtm.getnnz(1)>0]
dtm

<455282x6033 sparse matrix of type '<class 'numpy.int64'>'
	with 57260404 stored elements in Compressed Sparse Row format>

In [19]:
import lda

#n_topics = 30
#n_topics = 50
#n_topics = 70

n_topics = 25

topic_model = lda.LDA(n_topics=n_topics, n_iter=1500, random_state=1)

In [20]:
start = time.time()

document_topic_distributions = topic_model.fit_transform(dtm)

end = time.time()
print((end - start)/60.0,'minutes')

INFO:lda:n_documents: 455282
INFO:lda:vocab_size: 6033
INFO:lda:n_words: 85821427
INFO:lda:n_topics: 25
INFO:lda:n_iter: 1500
INFO:lda:<0> log likelihood: -999387143
INFO:lda:<10> log likelihood: -802106738
INFO:lda:<20> log likelihood: -727036001
INFO:lda:<30> log likelihood: -714965086
INFO:lda:<40> log likelihood: -711041390
INFO:lda:<50> log likelihood: -709297409
INFO:lda:<60> log likelihood: -708421660
INFO:lda:<70> log likelihood: -708009523
INFO:lda:<80> log likelihood: -707688464
INFO:lda:<90> log likelihood: -707479369
INFO:lda:<100> log likelihood: -707321404
INFO:lda:<110> log likelihood: -707195616
INFO:lda:<120> log likelihood: -707108559
INFO:lda:<130> log likelihood: -707024875
INFO:lda:<140> log likelihood: -706934412
INFO:lda:<150> log likelihood: -706875198
INFO:lda:<160> log likelihood: -706822219
INFO:lda:<170> log likelihood: -706759149
INFO:lda:<180> log likelihood: -706709297
INFO:lda:<190> log likelihood: -706702517
INFO:lda:<200> log likelihood: -706655422
INF

556.6239311854044 minutes


In [21]:
vocab = vectorizer.get_feature_names()
topic_names = ['Topic %d'%k for k in range(1, n_topics + 1)]

topic_word_distributions = pd.DataFrame(topic_model.components_, columns=vocab, index=topic_names)

document_topic_distributions = pd.DataFrame(document_topic_distributions,
                                            columns=topic_names,
                                            index=doc_ids[non_null_rows])

In [22]:
document_topic_distributions.to_csv('times_data/document_topic_distributions_'+str(n_topics)+'topics.csv')
topic_word_distributions.to_csv('times_data/topic_word_distributions_'+str(n_topics)+'topics.csv')

In [23]:
topic_word_distributions.loc['Topic 2'].sort_values(ascending=False).head(15)

court       0.029122
justice     0.017137
case        0.015390
lord        0.013637
law         0.013350
appeal      0.012037
act         0.011992
judge       0.008661
order       0.008224
section     0.007727
legal       0.007448
evidence    0.006885
judgment    0.005728
decision    0.005381
right       0.004980
Name: Topic 2, dtype: float64

In [24]:
topic_word_distributions.loc['Topic 3'].sort_values(ascending=False).head(15)

war           0.011055
president     0.010404
military      0.009081
soviet        0.008849
forces        0.007517
defence       0.006984
army          0.006935
american      0.006931
united        0.006644
government    0.006548
israel        0.005824
troops        0.005445
states        0.005442
foreign       0.005119
peace         0.004751
Name: Topic 3, dtype: float64

In [26]:
ls times_data

all_KLs_50topics.csv
all_KLs_per_year_50topics.csv
dates.txt
date_to_entropies.csv
date_to_n_issue.csv
document_topic_distributions_10topics.csv
document_topic_distributions_15topics.csv
document_topic_distributions_20topics.csv
document_topic_distributions_25topics.csv
document_topic_distributions_50topics.csv
dtm_matrix.npz
eff_n_topics_per_month.csv
eff_n_topics_per_trimester.csv
eff_n_topics_per_week.csv
eff_n_topics_per_year.csv
newspieces_per_month.csv
newspieces_per_trimester.csv
newspieces_per_year.csv
novelty_50topics_times.tar.gz
spacy_stopwords.txt
topic_word_distributions_10topics.csv
topic_word_distributions_15topics.csv
topic_word_distributions_20topics.csv
topic_word_distributions_25topics.csv
topic_word_distributions_50topics.csv
